In [1]:
!pip install uv -qU
!uv pip install unsloth tensorboard -qU --system


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
  × Failed to build `xformers==0.0.30`
  ├─▶ The build backend returned an error
  ╰─▶ Call to `setuptools.build_meta:__legacy__.build_wheel` failed (exit
      status: 1)

      [stderr]
      Traceback (most recent call last):
        File "<string>", line 14, in <module>
        File
      "/Users/pavankumartaddi/.cache/uv/builds-v0/.tmpvs91DB/lib/python3.11/site-packages/setuptools/build_meta.py",
      line 331, in get_requires_for_build_wheel
          return self._get_build_requires(config_settings, requirements=[])
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File
      "/Users/pavankumartaddi/.cache/uv/builds-v0/.tmpvs91DB/lib/python3.11/site-packages/setuptools/build_meta.py",
      line 301, in _get_build_requires
          self.run_setup()
        File
      "/Users/pavankumartaddi/.cache/uv/builds-v0/.tmpvs91DB/lib/python3.

In [2]:
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
os.environ["HF_HOME"] = "./"

In [3]:
from huggingface_hub import HfFolder, login
if HfFolder.get_token() is None:
  login()

ModuleNotFoundError: No module named 'huggingface_hub'

In [ ]:
model = "Qwen/Qwen3-8b"
max_seq_length = 8192
dtype  = None
load_in_4bit = False
load_in_8bit = False
datasets = "qwen3_finetune_dataset.jsonl"

In [ ]:
import torch
import gc, inspect, sys
def clear_old_model_refs():
  frm = inspect.currentframe().f_back
  caller_locals = frm.f_locals
  caller_globals = frm.f_globals
  for var in ("teacher", "model", "tokenizer"):
    if var in caller_locals:
      try:
        del caller_locals [var]
        if var in sys.modules: 
          del sys.modules [var]
        print(f"deleted local {var}")
      except Exception as e:
        print(f"could not delete local (var): {e}")

    if var in caller_globals:
      try:
        del caller_globals [var]
        print(f"deleted global (var)")
      except Exception as e:
        print(f"could not delete global (var): {e}")
  gc.collect()
  torch.cuda.empty_cache()
  print("GPIL cache cleared")

In [ ]:
clear_old_model_refs()
model,tokenizer = FastLanguageModel.from_pretrained(
    model_name = model,
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
    load_in_8bit = load_in_8bit,
    token = ""

)
print(tokenizer.padding_side)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

def unwrap_to_base(model):
    """Return the module that actually owns the decoder layers' list."""
    CANDIDATE_ATTRS = (
        "model",
        "Language model",
        "transformer",
        "gpt_neox",
        "backbone",
        "base model",
    )

    seen = set()
    queue = [model]

    while queue:
        m = queue.pop()
        if id(m) in seen:
            continue
        seen.add(id(m))

        if hasattr(m, "layers"):
            return m

        for attr in CANDIDATE_ATTRS:
            if hasattr(m, attr):
                queue.append(getattr(m, attr))

    raise AttributeError("Could not find decoder layers - print(dir(model)) to inspect manually.")


def print_shapes_of_first_layer(model):
    base_model = unwrap_to_base(model)
    first_layer = base_model.layers[0] if hasattr(base_model, "layers") else None

    if first_layer:
        for name, param in first_layer.named_parameters():
            print(f"{name}: {param.shape}")
    else:
        print("No layers found in the model.")


def visualize_matrix_dimensions(model_name='Qwen3-8B'):
    model = torch.load(model_name)
    param_shapes = {name: param.shape for name, param in model.named_parameters()}

    # Extract matrix dimensions
    matrix_dims = [(name, shape) for name, shape in param_shapes.items() if len(shape) == 2]
    matrix_dims.sort(key=lambda x: np.prod(x[1]), reverse=True)

    # Plotting
    fig, ax = plt.subplots(figsize=(12, 10))
    y_labels = [name for name, _ in matrix_dims]
    widths = [shape[1] for _, shape in matrix_dims]
    heights = [shape[0] for _, shape in matrix_dims]

    ax.barh(y_labels, widths, height=0.5, alpha=0.7, color='blue', label='Width')
    ax.barh(y_labels, heights, height=0.25, alpha=0.7, color='orange', label='Height')

    ax.set_xlabel("Dimensions")
    ax.set_title("Matrix Dimensions of Qwen3-8B")
    ax.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

# Example usage
# model = torch.load("path/to/qwen3-8b.pth")
# print_shapes_of_first_layer(model)
# visualize_matrix_dimensions(model)


In [ ]:
rank = 16
lora_alpha = 50
model = FastLanguageModel.get_peft_model(
    model,
    r = rank,
    finetune_vision_layers = False,
    finetune_language_layers = True,
    finetune_attention_modules = True,
    finetune_mlp_modules = True,
    use_gradient_checkpointing = "unsloth",
    full_finetuning = False,
    random_state = 3407,
    use_rslora = True
)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints trainable parameters and modules in a PyTorch model.
    Specifically useful for identifying LoRA adapter components.
    """
    trainable_params = 0
    all_param = 0
    trainable_modules = set()
    
    for name, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
            # Extract module name from parameter path
            module_name = ".".join(name.split(".")[:-1])
            trainable_modules.add(module_name)
    
    print(f"Trainable params: {trainable_params:,} | All params: {all_param:,} | Trainable%: {100 * trainable_params / all_param:.4f}%")
    print("\nTrainable modules:")
    for module in sorted(trainable_modules):
        print(f"- {module}")

In [ ]:
import torch
import math # Calculate training parameters
learning_rate = 5e^-5
per_device_train_batch_size  = 1
gradient_accumulation_steps = 4
epochs = 3
total_steps = (len(datasets) // (per_device_train_batch_size * gradient_accumulation_steps)) * epochs
warmup_steps = int(0.015 * total_steps)         # 1% of total steps for warmup
anneal_start_step = int(0.5 * total_steps)     # 50% of total steps for annealing start

# Print training configuration
print(f"Virtual batch size: {per_device_train_batch_size * gradient_accumulation_steps}")
print(f"Total steps: {total_steps}")
print(f"Warmup steps (rounded): {warmup_steps}")
print(f"Annealing start step: {anneal_start_step}")

# Initialize optimizer
from torch.optim import AdamW
optimizer = AdamW(
    model.parameters(),
    lr=learning_rate,
    weight_decay=0.01,
)

# Custom learning rate scheduler
from torch.optim.lr_scheduler import LambdaLR

def custom_cosine_scheduler(optimizer, num_warmup_steps, num_training_steps, anneal_start_step):
    """
    Custom scheduler with 3 phases:
    1. Linear warmup (first 1% of steps)
    2. Constant max LR (until 50% of steps)
    3. Cosine annealing decay (last 50% of steps)
    """
    def lr_lambda(current_step):
        # Warmup phase
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        
        # Constant phase
        elif current_step < anneal_start_step:
            return 1.0
        
        # Cosine annealing phase
        else:
            progress = (current_step - anneal_start_step) / \
                      (num_training_steps - anneal_start_step)
            return max(0.0, 0.5 * (1.0 + math.cos(math.pi * progress)))
    
    return LambdaLR(optimizer, lr_lambda)

# Create scheduler with cosine annealing
scheduler = custom_cosine_scheduler(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps,
    anneal_start_step=anneal_start_step,
)


In [ ]:
TEMPLATE = {
  "qwen" : (
    "<|im_start|>user\n",
    "<|im_start|>assistant\n"
  )
}

In [ ]:
## commom aargs
from unsloth import is_bfloat16_supported

common_args = {
"per_device_train_batch_size": per_device_train_batch_size,
"per_device_eval_batch_size": per_device_train_batch_size,
"gradient_accumulation_steps": gradient_accumulation_steps,
"num_train_epochs": epochs,
"logging_strategy": " steps",
"logging_dir": f"logs/{model.split('/')[-1]}",
"eval_strategy": "steps",
"logging_steps": min(max(int(0.05* total_steps), 1),10),
"eval_steps": max(int(0.1* total_steps), 1),
"bf16": is_bfloat16_supported(),
"fp16": not is_bfloat16_supported(),
"report_to": "tensorboard",
"seed": 3407,
"output_dir": "outputs",
"gradient_checkpointing": True,
"gradient_checkpointing_kwargs": {"use_reentrant": True},
"remove_unused_columns": True
}
common_args ["dataset_num_proc"] = 1
common_args["dataset_text_field"]= "text"
common_args ["max_seq_length"] = max_seq_length


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=ft_train_data,
    eval_dataset=ft_eval_data,
    args=TrainingArguments(**common_args),  # Convert dict to HF TrainingArguments

    packing=False,
)

# ===== Critical Unsloth Integration Section =====
# Manually inject custom optimizer and scheduler
trainer.optimizer = optimizer  # Your preconfigured optimizer
trainer.lr_scheduler = scheduler  # Your custom scheduler

# Prevent TRL from reinitializing components
trainer.create_optimizer = lambda *args, **kwargs: trainer.optimizer
trainer.create_scheduler = lambda *args, **kwargs: trainer.lr_scheduler
# ================================================

# Now you can safely call trainer.train()
trainer.train()